In [2]:
################################################################################################
# <문장 임베딩 예시>
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)측정 예시 3
# - contexts(문서)들를 sub 문장으로 나누고, 나눈 sub 문장들에 대해 토큰별로 나누어서 평균 임베딩벡터를 구해 Faiss 인덱싱 생성(contexts(문서)별 1개 Faiss 인덱싱 생성)
# - 쿼리문장 임베딩벡터를 구해서 각 contexts(문서) Faiss 인덱스에서 sub 문장 벡터와 비교하여 유사도 스코어 계산.
# - 말뭉치는  KorQuAD_v1.0 파일과 aihub(aihub.or.kr) 뉴스 기사 기계독해 데이터 QuA 파일 3종류 이용
#------------------------------------------------------------------------------
# 1. 검색모델 로딩
# 2. JSON 파일 로딩 후 df 만듬
# 3. 문장(문서)들을 sub 문장으로 분리 
# 4. 문단(문서)별 문장들의 토큰 평균들에 대해 Faiss 임베딩 생성
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# 6. MRR 계산
#-------------------------------------------------------------------------------
# 쿼리가:단어일때 검색율 (klue-sbert-v1 가 더좋음)
# -문장: 33%(쿼리문장:86.8%), 문장분리=47%, 문장+단어(50개)=53.60%/(70개)=59%, 128차원(50개)=50.40%/(100개)=55.8% =>결국 70개 768차원 벡터 생성이 가장 좋음(단어query: 59%, 문장query: 80%)
# -문장 토큰 고정 분할 : 5개분할=54.6%(쿼리문장:90%)/10개=46.6%(91%)/15개=41.2%/20개==40.6%(90.4%)/30개=38.60%
# -문장 토큰 가변 분할 : (5,7,8)=MAX_TOKEN_LEN:40개=52.8%(89.60%)/차원축소 128=>41.80%(81.8%)
# -BM25=Mecab 적용시: 85%, 적용안할때: 50%
#------------------------------------------------------------------------------------------------------------------------
# KorQuAD_v1.0_dev.json, DIM_RESIZE_LEN = 128, DIM_RESIZE_METHOD = 2, seed=111, search_k=5,
# EMBED_DIVIDE_LEN=[5,7,9], MAX_TOKEN_LEN=40
#------------------------------------------------------------------------------------------------------------------------
# 모델                    | IS_EMBED_DIVIDE|IS_ONNX_MODEL| 검출율 | 임베딩시간 |
#-----------------------------------------------------------------------------------------------------------------------
# klue-sbert-v1          | X              | X           | 26.2%  | 210초      
#-----------------------------------------------------------------------------------------------------------------------
# klue-sbert-v1         | 0              | X           | 43.8%  | 200초      
#-----------------------------------------------------------------------------------------------------------------------
# klue-sbert-v1-onnx   | x              | 0           | 16.4%  | 160초  
#-----------------------------------------------------------------------------------------------------------------------
# klue-sbert-v1-onnx  | 0              | 0           | 43%   | 175초  
#-----------------------------------------------------------------------------------------------------------------------
# kpf-sbert-v1.1-onnx  | 0             | 0         | 42%   | 175초  
#-----------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
# sklenarn 으로 cosine 확인 예제
# from sklearn.metrics.pairwise import cosine_similarity
# cosine = cosine_similarity([embed_querys[0]], [embed_querys_1[0]]) # (1,768) 식에 2차원 배열입력되어야 함.
#-------------------------------------------------------------------------------
################################################################################################

import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder

logger = mlogging(loggername="MRR-VOCAB", logfilename="../../../log/MRR-VOCAB")
device = GPU_info()
#device = 'cpu'  # cpu 테스트 할때

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 토큰 임베딩 관련 param
IS_EMBED_DIVIDE = True     # False=토큰임베딩 안함/True=문장에 대해 토큰별루 분리해서 평균구해서 임베딩 구함
EMBED_DIVIDE_LEN = [5,7,9]#[1,1,1]#[5,7,9] #5 # 문장을 몇개(토큰)으로 분리할지 (7,8,10) 일때 성능 좋음=>50.8%, (5,7,9) 일때 차원축소 128=>41.80%(81.8%) 성능 좋음
MAX_TOKEN_LEN = 40 #512 #40          # 최대 몇개 token까지만 임베딩 할지

# 차원 축소 관련 param
# 차원 축소 할지, 768 값을 앞에 384 만큼만 배열 resize 시킴.  
# - 384로 줄일대 -2% 성능 저하 발생(512: -1.2%, 256: -6%)
DIM_RESIZE_METHOD = 0  # 0=차원축소 안함/1=resize 로 차원축소/2=Dense 모델 이용 차원축소
DIM_RESIZE_LEN = 128

# ONNX 모델 사용(*무조건 cpu로 실행됨)
IS_ONNX_MODEL = False        # True=onnx 모델 사용

# 쿼리 임베딩 param
# - 쿼리 임베딩 안할때 보다도 성능이 떨어짐(문장검색 5%이상 떨어짐,단어검색은 유사함(10정도))
IS_QUERY_EMBED_DIVIDE = False  # True= 쿼리도 토큰으로 나누고 각 토큰별 max 값들 구한후 합해서 유사도 구함
QUERY_DIVIDE_LEN=10  # 쿼리를 몇개 토큰으로 나눠서 평균을 구할지 계수 설정(*10개 정도면 쿼리임베딩 안할때 단어검색일때 유사도 비슷하게 나옴)
#------------------------------------------------------------------------------------

# param 인자 범위 체크
if faiss_index_method > 1 or faiss_index_method < 0:
    raise ValueError(f"faiss_index_method = {faiss_index_method} is not bad!!")
    
seed_everything(seed)

logfilepath:../../../log/MRR-VOCAB_2023-03-08.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [ ]:
#------------------------------------------------------------------------------------------------
# 차원축소에 사용할 모델의 linear.weight, linear.bias 뽑아내서 저장??
#-------------------------------------------------------------------------------------------------
'''
import torch
from myutils import bi_encoder, dense_model, onnx_model, onnx_embed_text
from sentence_transformers import SentenceTransformer, models

bi_encoder_path = "../../../data11/model/moco/klue-aihub/klue-sbert-v1-aihub"  #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls

 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128 if DIM_RESIZE_METHOD == 2 else 0 if DIM_RESIZE_METHOD != 2 else None

word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=256, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
#bi_encoder1 = SentenceTransformer(bi_encoder_path)
#print(bi_encoder1)

state_dict = bi_encoder1.state_dict()  # bi_encoder모델이 state_dict 얻어옴
#print(state_dict.keys())
dense_weight = state_dict['2.linear.weight'] # denser 모델에 bi_encoder모델이 linear.weight 적용
dense_bias = state_dict['2.linear.bias']     # denser 모델에 bi_encoder모델이 linear.bias 적용
    
# 처음에  weigth, bias 파일을 저장해 둠.
torch.save(dense_weight, 'klue-sbert-v1-aihub-weight-128.pt')
torch.save(dense_bias, 'klue-sbert-v1-aihub-bias-128.pt')
'''

In [ ]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
import torch
from myutils import bi_encoder, dense_model, onnx_model, onnx_embed_text

bi_encoder_path = "bongsoo/kpf-sbert-v1.1" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls

 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128 if DIM_RESIZE_METHOD == 2 else 0 if DIM_RESIZE_METHOD != 2 else None

word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------------
# 차원축소
#------------------------------------------------------------------------------------------------------------------------
#  출력 값 차원 축소 지정인 경우, token_embeddings 일때는 sentencebert 라이브러리를 이용하여 dense_model 모델 추가할수 없으므로,
#  사용자정의 dense_model을 정의해서, 가중치와 bias를 bi_encoder모델에 것을 얻어와서 적용해서 차원 죽소함.
# - resize 방식 보다 성능 떨어지지만, 128일때는 더 성능이 좋음
# - weight와 bias 변경 해서 테스트시.klue-sbert-v1-weight-128 가장 성능 좋음(43.80%)
#  => klue-sbert-v1-weight-128.pt=43.80%, kpf-sbert-v1.1-weight-128=42%, 
#     all-mpnet-base-v2-bias-128/paraphrase-multilingual-mpnet-base-v2-bias-128=42.20%, bert-base-multilingual-uncased-bias-128=41.20% 
if DIM_RESIZE_METHOD == 2:
    dense_weight_path = './data/dense_weight/klue-sbert-v1-aihub-weight-128.pt'
    bias_weight_path = './data/dense_weight/klue-sbert-v1-aihub-bias-128.pt'
    
    #-------------------------------------------------------------------------
    # 처음에는 아래 코드를 활용하여 해당 모델의 128 weight와 bias를 저장해 두어야 함.
    #state_dict = bi_encoder1.state_dict()  # bi_encoder모델이 state_dict 얻어옴
    #print(state_dict.keys())
    #dense_weight = state_dict['2.linear.weight'] # denser 모델에 bi_encoder모델이 linear.weight 적용
    #dense_bias = state_dict['2.linear.bias']     # denser 모델에 bi_encoder모델이 linear.bias 적용
    
    # 처음에  weigth, bias 파일을 저장해 둠.
    #torch.save(dense_weight, dense_weight_path)
    #torch.save(dense_bias, bias_weight_path)
    #-------------------------------------------------------------------------
    # weigth, bias 저장해둔 파일 로딩
    dense_weight = torch.load(dense_weight_path)
    dense_bias = torch.load(bias_weight_path)

    print(f'\n---dense param---------------------------')   
    print('*dense_weight:{}({})'.format(dense_weight.size(), dense_weight_path))
    print(f'*dense_bias:{dense_bias.size()}({bias_weight_path})')
#------------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------------
# onnx 모델 로딩
#------------------------------------------------------------------------------------------------------------------------
if IS_ONNX_MODEL == True:
    onnx_model_path = "bongsoo/kpf-sbert-v1.1-onnx"#"../../../data11/model/onnx/klue-sbert-v1-onnx" #"bongsoo/klue-sbert-v1-onnx"
    onnx_tokenizer, onnx_model = onnx_model(onnx_model_path)
    print(f'\n---onnx_model---------------------------')
    print(onnx_model)
#------------------------------------------------------------------------------------------------------------------------

In [ ]:
#------------------------------------------------------------------------------------------------
# token_embeddings일때는 차원 축소롤 dense_model에 bi_encoder weight와 bias를 적용해서 처리??
#-------------------------------------------------------------------------------------------------
'''
from myutils import dense_model,embed_text

state_dict = bi_encoder.state_dict()
#print(state_dict.keys())

weigth = state_dict['2.linear.weight']
bias = state_dict['2.linear.bias']

embed_context = ['독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다']
token_embeds = embed_text(model=bi_encoder, paragraphs=embed_context, token_embeddings=True, return_tensor=False)
print(token_embeds[0].shape)
token_embed = dense_model(embed_tensor=token_embeds[0], out_features=128, weight=None, bias=None)
print(token_embed.size())

embed_context = ['독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다']
token_embeds = embed_text(model=bi_encoder, paragraphs=embed_context, token_embeddings=True, return_tensor=False)
print(token_embeds[0].shape)
token_embed2 = dense_model(embed_tensor=token_embeds[0], out_features=128, weight=None, bias=None)
print(token_embed2.size())

print(token_embed)
print()
print(token_embed2)

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(token_embed.detach().numpy(), token_embed2.detach().numpy()) # (1,768) 식에 2차원 배열입력되어야 함.
print(cosine_sim[1])
'''

In [ ]:
#--------------------------------------------------
# 테스트 같은 단어도 문장이 다르면 임베딩이 달라진다??
#--------------------------------------------------
'''
from myutils import embed_text
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

text = "배를 맛있게 먹다"
tmp = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=True)
print(tmp[0].size())
tmp1=tmp[0].cpu().numpy()
print(tmp1.shape)

tokenizer = word_embedding_model.tokenizer
token = tokenizer.tokenize(text)
print(token)

#tmp1 = [tmp[0].cpu().numpy().mean(axis=0)]
#embed1 = np.array(tmp1).astype('float32')
#embed1 = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=False)
#print(type(embed1))
#print(embed1.shape)

text = "배가 너무 달콤하다"
tmp = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=True)
print(tmp[0].size())
tmp2=tmp[0].cpu().numpy()
print(tmp2.shape)

tokenizer = word_embedding_model.tokenizer
token = tokenizer.tokenize(text)
print(token)

text = "배를 타고 여행을 간다"
tmp3 = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=True)
tt = [tmp3[0].cpu().numpy().mean(axis=0)]
tmp3 = np.array(tt).astype('float32')
print(tmp3.shape)


score = cosine_similarity([tmp1[1]], tmp3) # (1,768) 식에 2차원 배열입력되어야 함.
#score = util.semantic_search([tmp2[1]], tmp3, score_function=util.dot_score)

print(f'*유사도:{score}')
'''

In [4]:
#-------------------------------------------------------------------------------------------------------
# 2. JSON 파일 로딩 후 df 만듬
# => KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# list 데이터들을 dataframe으로 만듬
'''
contexts = [
        [1, '독도 독도 독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다'], 
        [11,'대구시는 11일 내년 정부 예산 3조1330억원을 확보해 전년보다 2%(611억원) 늘어났다고 밝혔다.그러나 올해 국비 증액 규모(1817억원)와 비교해 절반 수준에 그치며, 물산업클러스터 r&d 사업(200억), 국립청소년진로직업체험 수련원 건립 등은 한푼도 반영되지 않았다.'],
        [21,'자유한국당 (이름) 의원이 "공수처는 정권 유지를 위한 수단으로, 공수처가 있었으면 조국 수사를 못했을 것"이라고 주장했다.최 의원은 18일 대구 호텔수성에서 열린 대구 경북중견언론인모임 <아시아포럼21> 토론회에서 "검찰은 정권 말기가 되면 정권에 칼을 들이댔다. 공수처는 검찰의 칼끝을 회피하기 위한 것"이라며 이같이 말했다'],
        [2,'27일 오전 천연기념물(201-2호)이며 멸종위기 2급인 큰고니 떼가 경북 포항시 북구 흥해읍 샛강에 날아들었다.이날 관측된 큰고니는 어미 8마리와 새끼 3마리다.앞서 지난 19일 큰고니 8마리가 올들어 처음으로 관측됐다.큰고니들은 포항시 흥해읍 샛강에서 수초 등을 먹고 휴식한 후 대구 안심습지와 낙동강 하구 등지로 날아갈 것으로 보인다'],
        [3,'3일 대구와 경북지역은 구름이 많겠으며, 동해안과 북부 내륙에는 비가 내릴 것으로 예상된다.대구기상청에 따르면 중국 북동지방에서 남하하는 고기압의 가장자리에 들어 구름이 많겠으며, 경북 북부 내륙은 북쪽을 지나는 약한 기압골의 영향을 받아 오후부터 비가 약하게 내리거나 빗방울이 떨어지겠다.'],
        [31,'각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다'],
        [4,'26일 오후 11시44분쯤 대구 달서구 신당동의 한 아파트에서 원인 모를 불이 나 주민 1명이 다치고 수십명이 대피했다.27일 소방당국에 따르면 신고를 받고 소방차 24대와 소방대원 74명이 출동해 20분만에 불길을 잡았다.이 불로 아파트 2층에 사는 a씨(53)가 온 몸에 화상을 입어 병원으로 이송됐으며, 연기에 놀란 주민 50여명이 대피하는 소동을 빚었다.']
       ]
df_contexts = pd.DataFrame(contexts, columns=['contextid','context'])
print(df_contexts['context'].values)
'''

# KorQuAD_v1.0 혹은 aihub 뉴스 기사 기계독해 데이터 QuA 파일을 불러옴.
jsonfile = './data/KorQuAD_v1.0_dev.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_korquad_v1_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [5]:
#-------------------------------------------------------------------------------------------------------
# 3. 문장(문서)들을 sub 문장으로 분리
# =>df_contexts에 대해 각 문장별 '.'(마침표) 로 구분해서 sub 문장을 만듬. 혹은 kss 이용해서 sub 문장을 만듬
#-------------------------------------------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import split_sentences

#text = ["해외에서 데이터 무제한?? 뿐 아니라 음성과 문자 요금을 할인 받을 수 있는 ++요금제가 나왔다. 날씨는 좋다. lg유플러스는 중국, 일본, 홍콩, 싱가포르, 필리핀 등 아시아 8국을 대상으로~~ 무제한 데이터와 음성, 문자를 할인해주는 '스마트 로밍 요금제' 2종을 오는 28일부터 판매한다고 27일 밝혔다. . . 우선 '스마트 로밍음성'은 하루 기본료가 2000원으로 음성발신은 1분당 1000원이며, 문자메시지(sms)와 멀티미디어 문자메시지(mms)는 1건당 150원이다. 종전 500원에서 350원 인하했다. . . '스마트 로밍패키지'는 여기에 데이터 무제한 로밍서비스가 더해져 하루 기본료가 1만1000원이다. 기본료는 로밍 기간에 상관없이 사용한 당일에만 청구된다. . . 스마트 로밍 요금제는 일단 오는 7월 말까지 프로모션 형태로 제공하고 이후 정식 요금제로 추진한다는 계획이다. . . 아울러 해외에서 무제한 데이터 로밍 서비스와 데이터로밍 차단을 신청·해지할 수 있는 로밍 전용 모바일 홈페이지((이메일))도 운영한다. . . (이름) lg유플러스 글로벌로밍팀장은 '아시아 출(이름) 여행을 계획하고 있는 고객들이 저렴한 비용으로 안심하고 로밍 서비스를 사용할 수 있도록 이번 프로모션 요금제를 출시했다'며 '지속적으로 해외 로밍 이용 고객들의 편의성 증대를 위한상품을 준비할 예정'이라고 말했다. . . . . . . . # # #. . ■ 사진설명. . lg유플러스(부회장 (이름) / (이메일) )가 4월 28일부터 중국, 일본 등 아시아 8개국*을 대상으로 데이터와 음성 문자까지 할인해서 제공하는 스마트 로밍요금제(스마트 로밍음성/스마트 로밍패키지) 2종을 출시했다."]
#text =["오늘은 날씨가 ^^~~~~좋다. 내일은 비가 오고 춥겠다고 한다^^. 걱정이다. 오늘만큼만 매일 날씨가 좋으면 좋겠다.~~~~"]
sub_contexts = []
sentences = []

contexts = df_contexts['context'].values.tolist()
#contexts = text
start = time.time()
sub_contexts = split_sentences(paragraphs=contexts, sentences_split_num=10, paragraphs_num=999, debug=False)
logger.info(f'*문장 분리 시간 : {time.time()-start:.4f}')

print(sub_contexts[0])

  0%|          | 0/964 [00:00<?, ?it/s][Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 964/964 [00:13<00:00, 69.14it/s]
2023-03-08 16:45:25,274 - MRR-VOCAB - INFO - *문장 분리 시간 : 13.9487
[Kss]: *문장 분리 시간 : 13.9487


['1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다.', '1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다.', '같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다.', '경찰은 12월 18일20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다.', '1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다.', '임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.']


In [ ]:
from myutils import embed_text, onnx_embed_text

# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragrphs:list):
    if IS_ONNX_MODEL == True:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs) 
            
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs, token_embeddings=False)
    else:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, token_embeddings=True, return_tensor=False)
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, return_tensor=False)  
    
    return embeddings
   

In [ ]:
#-------------------------------------------------------------------------------------
# 4. 문단(문서)별 문장들의 토큰 평균들에 대해 Faiss 임베딩 생성
# - 각 문단의 문장들에 대해 토큰들의 평균값을 구하고, Faiss에 임베딩 추가함.
#-------------------------------------------------------------------------------------
import torch
from myutils import embed_text, dense_model, fassi_index, divide_arr_avg, divide_arr_avg_exten

EMBED_CONTEXTS = sub_contexts#sub_contexts #sub_vocab_contexts # 임베딩을 구해 인덱스에 추가할 문장들 1차원 리스트 (예:['오늘은 좋다','날씨가 흐리다','제주도 날씨',...])
faissindexlist = []

logger.info(f'*faiss 인덱싱 시작 => 방식: 토큰 분리:{IS_EMBED_DIVIDE}({EMBED_DIVIDE_LEN})/{faiss_index_method}(0=코사인, 1=유클리드)/차원축소:{DIM_RESIZE_METHOD}({DIM_RESIZE_LEN})/ONNX:{IS_ONNX_MODEL}')
    
start = time.time()
embed_context = []
embed_len = []
# EMBED_CONTEXTS 2차원 리스트 형태 : 예시 [['날씨가좋다','오후에는 비가온다','내일은 춥다'],['사과는맛있다','배도 맛있다'],[],[],...]
for idx, embed_context in enumerate(tqdm(EMBED_CONTEXTS)):
     
    # 임베딩 처리
    if IS_EMBED_DIVIDE == False:
        embeddings = embedding(embed_context)
    else:
        token_embeds = embedding(embed_context)
    
    #------------------------------------------------------------------------------------------------------------------------ 
    # 토큰 분할인 경우 처리 start=> 
    if IS_EMBED_DIVIDE == True:
        token_embed_arr_list = []
        tcount = 0
        # tensor(250,768) 한문장 토큰 임베딩 얻어와서, 각 ?개 토큰씩 평균을 구함.
        for token_embed in token_embeds:
            token_embed = token_embed[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
            
            if tcount >= MAX_TOKEN_LEN: 
                break
                '''
           # Dense 방식으로 차원 축소 => 평균 구하기 전에 차원 축소
            if DIM_RESIZE_METHOD == 2:
                token_embed = dense_model(embed_tensor=token_embed, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
                token_embed_arrs = token_embed.detach().numpy().astype('float32')
            else:
                # tensor를 arrary로 변경
                token_embed_arrs = token_embed.cpu().numpy().astype('float32')
                '''
            # tensor를 arrary로 변경
            token_embed_arrs = token_embed.cpu().numpy().astype('float32')
            
             # 7,8,10 씩 자르면서 문장 토큰 평균을 구함
            token_embed_divide_arrs = divide_arr_avg_exten(embed_arr=token_embed_arrs, divide_arrs=EMBED_DIVIDE_LEN, debug=False)
            #arrs = token_embed_divide_arrs
   
                    
            # Dense 방식으로 차원 축소 => 평균 구한후 차원 축소하는 방식이 0.6% 정도 성능 좋음
            if DIM_RESIZE_METHOD == 2:
                tmp1 = torch.Tensor(token_embed_divide_arrs)
                #tmp1 = torch.from_numpy(token_embed_divide_arrs)
                tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias, debug=False, idd=idx)                
                arrs = tmp2.detach().numpy().astype('float32')
      
            else:  
                arrs = token_embed_divide_arrs

            # 평균 구한 토큰들을 token_embed_arr_list 리스트에 담아둠.(50보다 크면 50개만 담음)      
            
            for idx, arr in enumerate(arrs):                  
                # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
                if DIM_RESIZE_METHOD == 1:
                    darr = np.resize(arr, (DIM_RESIZE_LEN,))
                else:
                    darr = arr
                
                token_embed_arr_list.append(darr)
                tcount +=1
                if tcount >=MAX_TOKEN_LEN:
                    break
                
        embeddings = np.array(token_embed_arr_list)
        # 토큰 분할인 경우 처리 End<== 
        #------------------------------------------------------------------------------------------------------------------------  
        
    embed_len.append(len(embeddings))
            
    # Faiss index 생성하고 추가 
    index = fassi_index(embeddings=embeddings, method=faiss_index_method)
    faissindexlist.append(index)
    
logger.info(f'*임베딩 시간 : {time.time()-start:.4f}')
logger.info(f'*임베딩 shape : {embeddings.shape}')

ecount = 0
for elen in embed_len:
    if elen >= MAX_TOKEN_LEN:
        ecount+=1
        
logger.info(f'*인덱스별 임베딩 수 : {embed_len}/*최대값:{max(embed_len)}/*{MAX_TOKEN_LEN} <= 계수:{ecount}')

In [ ]:
#-------------------------------------------------------------------------------------
# 쿼리문장 샘플링
#user_querys = ["독도에서 사고가 나서 실종자가 발생했다.", "오늘 날씨가 흐리고 비가 오겠다."]
#-------------------------------------------------------------------------------------
from myutils import df_sampling

# 쿼리 샘플링함.
if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_querys = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_querys = df_questions['question'].values.tolist()
  
print(f'Query-----------------------------------------------------')
print(user_querys[0:3])

In [ ]:
#-------------------------------------------------------------------------------------
# 5.쿼리 후 단어 MAX 스코어 합 구함
# => 쿼리 문장을 각 토큰별 임베딩값을 구하고, 이후 Faiss 문장 임베딩과 비교하여 가장 유사한 문장을 찾음
# => 3,000 문장 GPU 환경에서 약 4분 걸림.
#-------------------------------------------------------------------------------------

if IS_QUERY_EMBED_DIVIDE == True:
    
    from myutils import df_sampling, sum_of_array_2d, index_of_list,split_sentence_list, clean_text

    bi_predictions_list = []

    for i, user_query in tqdm(enumerate(user_querys)):

        if i < 5:
            print(user_query)

        # 쿼리별 토큰 벡터를 구함 
        vectors_tmp = embed_text(model=bi_encoder1, paragraphs=[user_query], token_embeddings=True, return_tensor=False)
        vectors_tmp2 = vectors_tmp[0].cpu().numpy().astype('float32') # tensor 토큰 벡터를 np.array로 변경
        vectors_tmp2 = vectors_tmp2[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
        
        vectors = divide_arr_avg(embed_arr=vectors_tmp2, divide_len=QUERY_DIVIDE_LEN) # QUERY_DIVIDE_LEN 만큼 자르면서 쿼리 토큰 평균을 구함

        # *cosine유사도 구할때는 반드시 normalize 처리함.
        if faiss_index_method == 0:
            faiss.normalize_L2(vectors)          

        max_values_list = []

        # 기존 contexts에 대한 faiss 문장 인덱스를 불러오면서, 단어 별루 쿼리 검색 해서 스코어를 구하고, 구한 단어별 스코어를 모두 더해서 총합을 구함.
        for count, index in enumerate(faissindexlist):

            # 각 faiss 문장 인덱스에서 문장단어 쿼리 검색 함 (distance, idx= numpy.array 타입임)
            distance, idx = index.search(np.array(vectors).astype("float32"), k=3) # 단어별 검색해서 3개 스코어를 얻어옴.

            # 예) distance = np.array([[2,3,4],[4,3,2]])
            # faiss_index_method=0 => cosine유사도 구할때는 max 값을 찾음, 1=>유클리드 거리 로 할때는 min 값을 찾음
            sum_value = sum_of_array_2d(array=distance, bmin=faiss_index_method)

            # 각 문장별 스코어 최대값을 저장해 둠.
            max_values_list.append(sum_value)

        # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
        # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
        # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
        indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  

        # 예측검색결과 contextid값들을 리스트로 만듬.
        tmp_bi_predictions_list = []
        for indice in indices:
            tmp_bi_predictions_list.append(df_contexts["contextid"][indice])

        # 2D 예측검색결과 리스트에 추가 
        bi_predictions_list.append(tmp_bi_predictions_list)

    print(bi_predictions_list[0:3])

In [ ]:
#-------------------------------------------------------------------------------------
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# => 쿼리 문장에 임베딩값을 구하고, 이후 Faiss sub 문장 임베딩과 비교하여 가장 유사한 문장 avg_num개 를 찾고 평균을 구함
#    이후 평균값이 가장 큰 search_k 갯수 만 쿼리에 대한 예측 결과 리스트로 만듬
#-------------------------------------------------------------------------------------

if IS_QUERY_EMBED_DIVIDE == False:
    
    from myutils import df_sampling, sum_of_array_2d, index_of_list,split_sentence_list, clean_text

    bi_predictions_list = []

    # 임베딩 처리
    if IS_EMBED_DIVIDE == False:
        embed_querys = embedding(user_querys)
    else:
        token_query_embeds = embedding(user_querys)

    if IS_EMBED_DIVIDE == True:
        token_query_embed_arr_list = []
        # 쿼리 문장들의 토큰들의 평균을 구함.
        for token_query_embed in token_query_embeds:
            token_query_embed = token_query_embed[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌

            '''
            # Dense 방식으로 차원 축소 => 평균 구하기 전에 차원 축소
            if DIM_RESIZE_METHOD == 2:
                token_query_embed = dense_model(embed_tensor=token_query_embed, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
                tmp = token_query_embed.detach().numpy().astype('float32')
            else:
                tmp = token_query_embed.cpu().numpy().astype('float32')
                '''
            tmp=token_query_embed.cpu().numpy().astype('float32')
            tmp=tmp.mean(axis=0) #평균 구함(768,) 1차원배열로 출력됨

            # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
            if DIM_RESIZE_METHOD == 1:
                tmp = np.resize(tmp, (DIM_RESIZE_LEN,))
             # Dense 방식으로 차원 축소=> 평균 구하기 전에 차원 축소하는것이 성능 +0.6더 좋음
            elif DIM_RESIZE_METHOD == 2:
                tmp1 = torch.Tensor([tmp]) # 1차원 배열을 -> 2차둰 텐서(1,768)로 변환
                tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
                tmp = tmp2.detach().numpy().astype('float32').ravel(order='C') # 1차원 배열로 변경(128,)            

            token_query_embed_arr_list.append(tmp)

        embed_querys = np.array(token_query_embed_arr_list)  # 리스트를 배열로 변환  
    #------------------------------------------------------------------


    if faiss_index_method == 0:
        faiss.normalize_L2(embed_querys)          # *cosine유사도 구할때는 반드시 normalize 처리함.

    for embed_query in tqdm(embed_querys):
        embed_query = [embed_query]

        max_values_list = []
        for count, index in enumerate(faissindexlist):
            distance, idx = index.search(np.array(embed_query).astype("float32"), k=avg_num) # avg_num 계수 만큼 유사한 sub문장을 찾음
            avg_distance = distance.mean(axis=1) # 검색된 sub 문장들에 대해 평균을 구함
            max_values_list.append(avg_distance[0])

        # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
        # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
        # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
        indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  

        # 예측검색결과 contextid값들을 리스트로 만듬.
        tmp_bi_predictions_list = []
        for indice in indices:
            tmp_bi_predictions_list.append(df_contexts["contextid"][indice])
            #print(f'*인덱스:{indice}/contextid:{df_contexts["contextid"][indice]}-----------------------------------------------------------')
            #print('*총합/평균 : {:.4f}/{:.4f}'.format(max_values_list[indice], float(max_values_list[indice]/len(mecab_query))))
            #print(df_contexts['context'][indice])
            #print(mecab_contexts[indice])

        # 2D 예측검색결과 리스트에 추가 
        bi_predictions_list.append(tmp_bi_predictions_list)

    print(bi_predictions_list[0:3])


In [ ]:
#--------------------------------------------------------------------------------------------------
# 6. MRR 계산
# => 정답 리스트[2,3,1,4] 과 예측검색리스트[[1,2,5,1],[3,4,2,1],[6,5,4,1], [2,3,4,1]]를 입력하여 MRR 스코어 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'faiss 인덱싱 방식: 토큰 분리:{IS_EMBED_DIVIDE}({EMBED_DIVIDE_LEN})/{faiss_index_method}(0=코사인 유사도, 1=유클리드 거리)/차원축소:{DIM_RESIZE_METHOD}({DIM_RESIZE_LEN})/ONNX:{IS_ONNX_MODEL}')
logger.info('*avg_num:{}/search_k:{}/query_num:{}/out_dimension:{}'.format(avg_num, search_k, query_num, out_dimension))

# MRR 계산
bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, bi_predictions_list)

# BI-MRR 출력
logger.info(f'----------------------------------------------------------------------------')
logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
logger.info('*BI-MRR:{:.4f}'.format(bi_score))
logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
# 검색 한 계슈
#logger.info(f'---------------------------------------------------------------------------')
search_count = 0
for item in bi_ranks:
    if item != 0:
        search_count += 1
    
logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
logger.info(f'---------------------------------------------------------------------------')